**RIVM NO$_x$ measurements couple with station location**

In this notebook, I plotted NO$_x$ measurements across the Netherlands. I began by identifying the dataset's temporal and spatial details, ensuring it included values, timestamps, and station identifiers. I focused on stations with data from both 2019 and 2023 by comparing station IDs or names between the two years. I defined day and night intervals (day: 6 AM–7 PM, night: 7 PM–6 AM) and calculated average NO$_x$ values for 2019 day, 2019 night, 2023 day, and 2023 night. Optionally, I computed averages for weekdays if time allowed. I matched stations with latitude and longitude data, enriching the dataset if necessary. 

Finally, I prepared the data for import into ArcGIS Pro by converting it to an Excel file. 

I loaded the data into ArcGIS Pro, applied appropriate symbology to visualize NO$_x$ levels. (Not included in this notebook)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame
from pandas import Grouper
from pandas import read_csv
pd.options.mode.chained_assignment = None  # default='warn'

In [39]:
import geopandas as gpd
from shapely.geometry import Point

In [3]:
# Create a excel file with all NOx measurements stations and average concentrations.
# Read the files, NOx 2019, NOx 2023, station info,
NOx_2019=pd.read_csv("/Data/2019_NOx_breed.csv",sep=';',skiprows=5,encoding='unicode_escape')#read 2019 file
NOx_2023=pd.read_csv("/Data/2023_NOx_breed.csv",sep=';',skiprows=5,encoding='unicode_escape')#read 2023 files
Station_info=pd.read_csv("/Data/RIVM_StationLocations.csv",sep=';',skiprows=5,encoding='unicode_escape')#read location information file

# Edit the column name " begindatumtijd" to "begindatumtijd", change it to "datetime" format 
# Strip spaces from all column names
NOx_2019.columns = NOx_2019.columns.str.strip()
NOx_2023.columns = NOx_2023.columns.str.strip()
NOx_2019['begindatumtijd'] = pd.to_datetime(NOx_2019['begindatumtijd'], format='%Y-%m-%dT%H:%M:%S%z')
NOx_2019['begindatumtijd'] = pd.to_datetime(NOx_2019['begindatumtijd']).dt.tz_localize(None)
NOx_2023['begindatumtijd'] = pd.to_datetime(NOx_2023['begindatumtijd'], format='%Y-%m-%dT%H:%M:%S%z')
NOx_2023['begindatumtijd'] = pd.to_datetime(NOx_2023['begindatumtijd']).dt.tz_localize(None)

In [5]:
# Select the period of October for each NOx datasets (month number 10)
NOx_2019_Oct = NOx_2019[NOx_2019['begindatumtijd'].dt.month == 10]
NOx_2023_Oct = NOx_2023[NOx_2023['begindatumtijd'].dt.month == 10]

In [7]:
# Remove columns with more than 50 NaN values, so only almost complete measurements are there.
def remove_columns_with_many_nans(df, threshold=10, verbose=True):
    # Initial column count
    initial_columns = len(df.columns)
    
    # Count NaN values in each column
    nan_counts = df.isna().sum()
    
    # Get columns to drop
    columns_to_drop = nan_counts[nan_counts > threshold].index.tolist()
    
    # Remove these columns
    df_cleaned = df.drop(columns=columns_to_drop)
    
    if verbose:
        print(f"\nInitial number of columns: {initial_columns}")
        print(f"Columns removed: {len(columns_to_drop)}")
        print(f"Remaining columns: {len(df_cleaned.columns)}")
        
        if columns_to_drop:
            print("\nRemoved columns and their NaN counts:")
            for col in columns_to_drop:
                print(f"- {col}: {nan_counts[col]} NaN values")
                
        print("\nPercentage of columns retained: {:.1f}%".format(
            (len(df_cleaned.columns) / initial_columns) * 100))
    
    return df_cleaned

# Apply the function to both datasets
print("Analysis of October 2019 dataset:")
NOx_2019_Oct_cleaned = remove_columns_with_many_nans(NOx_2019_Oct, threshold=50)

print("\nAnalysis of October 2023 dataset:")
NOx_2023_Oct_cleaned = remove_columns_with_many_nans(NOx_2023_Oct, threshold=50)

Analysis of October 2019 dataset:

Initial number of columns: 91
Columns removed: 6
Remaining columns: 85

Removed columns and their NaN counts:
- NL01485_NOx_lucht: 744 NaN values
- NL01491_NOx_lucht.1: 744 NaN values
- NL01494_NOx_lucht: 217 NaN values
- NL01494_NOx_lucht.1: 531 NaN values
- NL10246_NOx_lucht: 313 NaN values
- NL49019_NOx_lucht: 72 NaN values

Percentage of columns retained: 93.4%

Analysis of October 2023 dataset:

Initial number of columns: 91
Columns removed: 5
Remaining columns: 86

Removed columns and their NaN counts:
- NL01493_NOx_lucht.1: 744 NaN values
- NL01497_NOx_lucht.1: 744 NaN values
- NL10633_NOx_lucht: 81 NaN values
- NL10807_NOx_lucht.1: 744 NaN values
- NL10929_NOx_lucht: 85 NaN values

Percentage of columns retained: 94.5%


In [9]:
# Find the stations that exist in both datasets, and exclude stations that do not have measurements at both years.
# Define a function to compare column sets.
def compare_column_sets(df1, df2, name1='Dataset 1', name2='Dataset 2', verbose=True):
    """
    Compare columns between two dataframes and identify common and unique columns.
    
    Parameters:
    df1 (DataFrame): First DataFrame
    df2 (DataFrame): Second DataFrame
    name1 (str): Name of first DataFrame for output
    name2 (str): Name of second DataFrame for output
    verbose (bool): If True, prints the results
    
    Returns:
    dict: Dictionary containing common and unique columns
    """
     # Convert columns to sets
    columns1 = set(df1.columns)
    columns2 = set(df2.columns)
    
    # Find common and unique columns
    common_columns = columns1.intersection(columns2)
    unique_to_df1 = columns1 - columns2
    unique_to_df2 = columns2 - columns1
    
    # Store results in a dictionary
    results = {
        'common_columns': sorted(list(common_columns)),
        f'unique_to_{name1}': sorted(list(unique_to_df1)),
        f'unique_to_{name2}': sorted(list(unique_to_df2))
    }
    
    # Print results if verbose is True
    if verbose:
        print(f"Common columns in both datasets ({len(common_columns)}):")
        print(results['common_columns'])
        print(f"\nColumns unique to {name1} ({len(unique_to_df1)}):")
        print(results[f'unique_to_{name1}'])
        print(f"\nColumns unique to {name2} ({len(unique_to_df2)}):")
        print(results[f'unique_to_{name2}'])
        
        # Print summary statistics
        print(f"\nSummary:")
        print(f"Total columns in {name1}: {len(columns1)}")
        print(f"Total columns in {name2}: {len(columns2)}")
        print(f"Common columns: {len(common_columns)}")
        
    return results
# Get the list of column names from both DataFrames
results = compare_column_sets(
    NOx_2019_Oct_cleaned, 
    NOx_2023_Oct_cleaned,
    name1='2019',
    name2='2023'
)

Common columns in both datasets (78):
['NL01487_NOx_lucht', 'NL01488_NOx_lucht', 'NL01489_NOx_lucht', 'NL01491_NOx_lucht', 'NL01493_NOx_lucht', 'NL01495_NOx_lucht', 'NL01496_NOx_lucht', 'NL01912_NOx_lucht', 'NL10107_NOx_lucht', 'NL10131_NOx_lucht', 'NL10133_NOx_lucht', 'NL10136_NOx_lucht', 'NL10138_NOx_lucht', 'NL10230_NOx_lucht', 'NL10235_NOx_lucht', 'NL10236_NOx_lucht', 'NL10237_NOx_lucht', 'NL10240_NOx_lucht', 'NL10241_NOx_lucht', 'NL10247_NOx_lucht', 'NL10301_NOx_lucht', 'NL10318_NOx_lucht', 'NL10404_NOx_lucht', 'NL10418_NOx_lucht', 'NL10437_NOx_lucht', 'NL10442_NOx_lucht', 'NL10444_NOx_lucht', 'NL10445_NOx_lucht', 'NL10446_NOx_lucht', 'NL10449_NOx_lucht', 'NL10538_NOx_lucht', 'NL10550_NOx_lucht', 'NL10617_NOx_lucht', 'NL10636_NOx_lucht', 'NL10639_NOx_lucht', 'NL10641_NOx_lucht', 'NL10643_NOx_lucht', 'NL10644_NOx_lucht', 'NL10722_NOx_lucht', 'NL10738_NOx_lucht', 'NL10741_NOx_lucht', 'NL10742_NOx_lucht', 'NL10807_NOx_lucht', 'NL10818_NOx_lucht', 'NL10918_NOx_lucht', 'NL10934_NOx_luc

In [11]:
# The column NL01485_NOx_lucht.1 in 2019 dataset can be renamed and is the same as "NL01485_NOx_lucht" in 2023.
NOx_2019_Oct_cleaned = NOx_2019_Oct_cleaned.rename(columns={'NL01485_NOx_lucht.1': 'NL01485_NOx_lucht'})

# Use the function again and delete unique station measurements.
results_new = compare_column_sets(
    NOx_2019_Oct_cleaned, 
    NOx_2023_Oct_cleaned,
    name1='2019',
    name2='2023'
)

# Get common columns from the results dictionary
common_columns = results_new['common_columns']

# Create new datasets with only common columns
NOx_2019_Oct_Updated = NOx_2019_Oct_cleaned[common_columns].copy()
NOx_2023_Oct_Updated = NOx_2023_Oct_cleaned[common_columns].copy()

# Remove '_NOx_lucht' from column names
NOx_2019_Oct_Updated.columns = [col.replace('_NOx_lucht', '') for col in NOx_2019_Oct_Updated.columns]
NOx_2023_Oct_Updated.columns = [col.replace('_NOx_lucht', '') for col in NOx_2023_Oct_Updated.columns]

Common columns in both datasets (79):
['NL01485_NOx_lucht', 'NL01487_NOx_lucht', 'NL01488_NOx_lucht', 'NL01489_NOx_lucht', 'NL01491_NOx_lucht', 'NL01493_NOx_lucht', 'NL01495_NOx_lucht', 'NL01496_NOx_lucht', 'NL01912_NOx_lucht', 'NL10107_NOx_lucht', 'NL10131_NOx_lucht', 'NL10133_NOx_lucht', 'NL10136_NOx_lucht', 'NL10138_NOx_lucht', 'NL10230_NOx_lucht', 'NL10235_NOx_lucht', 'NL10236_NOx_lucht', 'NL10237_NOx_lucht', 'NL10240_NOx_lucht', 'NL10241_NOx_lucht', 'NL10247_NOx_lucht', 'NL10301_NOx_lucht', 'NL10318_NOx_lucht', 'NL10404_NOx_lucht', 'NL10418_NOx_lucht', 'NL10437_NOx_lucht', 'NL10442_NOx_lucht', 'NL10444_NOx_lucht', 'NL10445_NOx_lucht', 'NL10446_NOx_lucht', 'NL10449_NOx_lucht', 'NL10538_NOx_lucht', 'NL10550_NOx_lucht', 'NL10617_NOx_lucht', 'NL10636_NOx_lucht', 'NL10639_NOx_lucht', 'NL10641_NOx_lucht', 'NL10643_NOx_lucht', 'NL10644_NOx_lucht', 'NL10722_NOx_lucht', 'NL10738_NOx_lucht', 'NL10741_NOx_lucht', 'NL10742_NOx_lucht', 'NL10807_NOx_lucht', 'NL10818_NOx_lucht', 'NL10918_NOx_luc

In [13]:
# delete columns "component eenheid einddatumtijd matrix meetduur", and set "begindatumtijd" as the key.
# List of columns to remove
columns_to_remove = ['component', 'eenheid', 'einddatumtijd', 'matrix', 'meetduur']

# Remove specified columns from 2019 dataset
NOx_2019_Oct_Updated = NOx_2019_Oct_Updated.drop(columns=columns_to_remove, errors='ignore')
# Set begindatumtijd as index for 2019
NOx_2019_Oct_Updated.set_index('begindatumtijd', inplace=True)

# Remove specified columns from 2023 dataset
NOx_2023_Oct_Updated = NOx_2023_Oct_Updated.drop(columns=columns_to_remove, errors='ignore')
# Set begindatumtijd as index for 2023
NOx_2023_Oct_Updated.set_index('begindatumtijd', inplace=True)


In [43]:
# Calculate the average for night and day (Oct 2019 day, Oct 2019 night, Oct 2023 day, Oct 2023 night, extra time: weekdays?).
# 2019 Calculations
# Create hour column
NOx_2019_Oct_Updated['hour'] = NOx_2019_Oct_Updated.index.hour

# Calculate day averages (6-19)
day_mask_2019 = (NOx_2019_Oct_Updated['hour'] >= 6) & (NOx_2019_Oct_Updated['hour'] < 19)
day_avg_2019 = NOx_2019_Oct_Updated[day_mask_2019].drop('hour', axis=1).mean()

# Calculate night averages (19-6)
night_mask_2019 = (NOx_2019_Oct_Updated['hour'] >= 19) | (NOx_2019_Oct_Updated['hour'] < 6)
night_avg_2019 = NOx_2019_Oct_Updated[night_mask_2019].drop('hour', axis=1).mean()

# 2023 Calculations
# Create hour column
NOx_2023_Oct_Updated['hour'] = NOx_2023_Oct_Updated.index.hour

# Calculate day averages (6-19)
day_mask_2023 = (NOx_2023_Oct_Updated['hour'] >= 6) & (NOx_2023_Oct_Updated['hour'] < 19)
day_avg_2023 = NOx_2023_Oct_Updated[day_mask_2023].drop('hour', axis=1).mean()

# Calculate night averages (19-6)
night_mask_2023 = (NOx_2023_Oct_Updated['hour'] >= 19) | (NOx_2023_Oct_Updated['hour'] < 6)
night_avg_2023 = NOx_2023_Oct_Updated[night_mask_2023].drop('hour', axis=1).mean()

# Create a new table containing stations with average concentration in 2019 and 2023.

# Create results DataFrame
NOx_Average = pd.DataFrame({
    'Day_Ave_19': day_avg_2019,
    'Nig_Ave_19': night_avg_2019,
    'Day_Ave_23': day_avg_2023,
    'Nig_Ave_20': night_avg_2023
})

# Print results
print("Results for all stations:")
print(NOx_Average)

# Clean up by dropping the hour columns we created
NOx_2019_Oct_Updated.drop('hour', axis=1, inplace=True)
NOx_2023_Oct_Updated.drop('hour', axis=1, inplace=True)

Results for all stations:
         Day_Ave_19  Nig_Ave_19  Day_Ave_23  Nig_Ave_20
NL01485   38.571596   28.402539   26.934156   20.122554
NL01487   89.011058   51.032610   59.986052   42.083053
NL01488   32.655757   25.596422   21.975444   19.552525
NL01489   60.724712   38.006628   39.580769   27.988258
NL01491   54.671077   37.229390   42.920412   34.651493
...             ...         ...         ...         ...
NL53004   29.759102   27.106745   36.018481   41.520882
NL53015   22.370426   17.693030   38.747558   45.635484
NL53016   24.175000   18.535758   14.304859   12.175367
NL53020   23.997215   20.907038   19.592072   16.007122
NL54004   58.437221   31.377067   37.438246   26.031601

[73 rows x 4 columns]


In [45]:
# Connect stations with lat and lon 
# Select and rename columns
Station_Lat_Lon = Station_info[['meetlocatie_id', 'breedtegraad', 'lengtegraad']].rename(
    columns={
        'meetlocatie_id': 'ID',
        'breedtegraad': 'Latitude',
        'lengtegraad': 'Longitude'
    }
)

In [47]:
# Set 'ID' as index for Station_Lat_Lon
Station_Lat_Lon = Station_Lat_Lon.set_index('ID')

# Perform inner join using merge
NOx_Average_with_Location = pd.merge(
    NOx_Average, 
    Station_Lat_Lon, 
    left_index=True, 
    right_index=True, 
    how='inner'
)

# Print results to verify
print("Shape of joined dataframe:", NOx_Average_with_Location.shape)
print("\nColumns:", NOx_Average_with_Location.columns.tolist())
print("\nSample of joined data:")
print(NOx_Average_with_Location.head())

Shape of joined dataframe: (73, 6)

Columns: ['Day_Ave_19', 'Nig_Ave_19', 'Day_Ave_23', 'Nig_Ave_20', 'Latitude', 'Longitude']

Sample of joined data:
         Day_Ave_19  Nig_Ave_19  Day_Ave_23  Nig_Ave_20   Latitude  Longitude
NL01485   38.571596   28.402539   26.934156   20.122554  51.867411   4.355242
NL01487   89.011058   51.032610   59.986052   42.083053  51.891147   4.480690
NL01488   32.655757   25.596422   21.975444   19.552525  51.893617   4.487528
NL01489   60.724712   38.006628   39.580769   27.988258  51.869431   4.580058
NL01491   54.671077   37.229390   42.920412   34.651493  51.938472   4.430692


In [51]:
# Basic way to save to Excel
NOx_Average_with_Location.to_excel('/Users/zhiyuwu/Desktop/GRS-35306/GroupProject/Data/NOx_Average_with_Location.xlsx')